In [47]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [48]:
NVH_metrics = ['6208 Cage Failure', 'CH/Input Bearing Race Defect', 'CH/Input Bearing Ball Defect',
						'CH Inter Bearing Race Defect',
						'CH Inter Bearing Ball Defect',
						'Input Gear Clicking',
						'Intermediate Gear Clicking',
						'Output Gear Clicking',
						'Intermediate Gear Clicking Negative',
						'Intermediate Gear Clicking Positive',
						'Input Gear Clicking Positive',
						'Input Gear Clicking Negative',
						'Output Gear Clicking Negative',
						'Output Gear Clicking Positive', 
                        'Motor 3rd Ord Avg', 
                        'Growl 4-6']

In [49]:
df_raw = pd.read_csv('datasets\Week24_DynoResults.csv')
df_raw.head()

,testID,isTestPass,part,FX_Thing,partnumber,testTime,testMonth,testDay,testYear,testHour,testMinute,metricName,metricispass,metricValue,yearWeek
0,253662,1,P1025598-01-U-ST14J0065840R2,P1025598-01-U:ST14J0065840R2,1025598-01-U,2024-06-10 05:37:17.000,6,10,2024,5,37,6208 Cage Failure,1,56.543930,24
1,253662,1,P1025598-01-U-ST14J0065840R2,P1025598-01-U:ST14J0065840R2,1025598-01-U,2024-06-10 05:37:17.000,6,10,2024,5,37,Max,1,554.949951,24
2,253662,1,P1025598-01-U-ST14J0065840R2,P1025598-01-U:ST14J0065840R2,1025598-01-U,2024-06-10 05:37:17.000,6,10,2024,5,37,Torque Inverter Side,1,317.578400,24
3,253663,1,P1025598-01-U-ST14J0065840R2,P1025598-01-U:ST14J0065840R2,1025598-01-U,2024-06-10 05:38:58.000,6,10,2024,5,38,Motor 1st Ord Max,1,0.192053,24
4,253663,1,P1025598-01-U-ST14J0065840R2,P1025598-01-U:ST14J0065840R2,1025598-01-U,2024-06-10 05:38:58.000,6,10,2024,5,38,Motor 2nd Order Max,1,0.160541,24


In [50]:
summary = pd.Series({'FirstPasses':0,
                     'DLE':0,
                           'Retest Success':0,
                           'Reworked Pass ':0,
                           'Pending Rework / Unsuccessful Rework':0,
                           'Machine Health':0
                           })

In [51]:
df = df_raw.copy()
#df.groupby('FX_Thing')['metricispass'].sum().min()
#if a test has 46 metricispass, it's a first time pass
#dropping Golden units
golden_units = ['P1025598-00-U:ST13D0013400', 'P1025598-00-U:ST17A1021978']
df = df[~df['FX_Thing'].isin(golden_units)]
retestTime = 20


In [52]:
#key and datetime
df['thing+metricName'] = df['FX_Thing']+'+'+df['metricName']
df['testTime'] = pd.to_datetime(df['testTime'])

In [53]:
df['part'].nunique()

111

In [54]:
#Finding all the first time passes. Every unit with 46 metricispass sum is a first time pass
sum_metricispass = df.groupby('FX_Thing')['metricispass'].sum()
firstPasses = sum_metricispass[sum_metricispass == 46].reset_index()['FX_Thing']
summary['FirstPasses'] = len(firstPasses)

In [55]:
#getting all the failures and the unique failing serials
df_allFailures = df[df['metricispass'] == 0].copy()
df_allFailures.head()

#count of unique failures
failing_serials = df_allFailures['FX_Thing'].drop_duplicates()
len(failing_serials)



10

In [56]:
#anything with only passes, but doesn't have 46 runs, is a DLE
DLEs = df['FX_Thing'].nunique() - len(firstPasses) - len(failing_serials)
summary['DLE'] = DLEs

In [57]:
#DLEs
uniqueSerials = df.drop_duplicates(subset = 'FX_Thing')
uniqueSerials[(~uniqueSerials['FX_Thing'].isin(failing_serials)) & (~uniqueSerials['FX_Thing'].isin(firstPasses))]

,testID,isTestPass,part,FX_Thing,partnumber,testTime,testMonth,testDay,testYear,testHour,testMinute,metricName,metricispass,metricValue,yearWeek,thing+metricName
0,253662,1,P1025598-01-U-ST14J0065840R2,P1025598-01-U:ST14J0065840R2,1025598-01-U,2024-06-10 05:37:17,6,10,2024,5,37,6208 Cage Failure,1,56.543930,24,P1025598-01-U:ST14J0065840R2+6208 Cage Failure
690,253707,1,P1025598-01-U-ST16C1004720,P1025598-01-U:ST16C1004720,1025598-01-U,2024-06-10 09:25:42,6,10,2024,9,25,6208 Cage Failure,1,52.874680,24,P1025598-01-U:ST16C1004720+6208 Cage Failure
3987,253926,1,P1025598-00-U-ST14E0047037R1,P1025598-00-U:ST14E0047037R1,1025598-00-U,2024-06-13 06:53:21,6,13,2024,6,53,6208 Cage Failure,1,54.003288,24,P1025598-00-U:ST14E0047037R1+6208 Cage Failure
4229,253948,1,P1025598-01-U-ST16L1021252,P1025598-01-U:ST16L1021252,1025598-01-U,2024-06-13 08:44:07,6,13,2024,8,44,6208 Cage Failure,1,54.470745,24,P1025598-01-U:ST16L1021252+6208 Cage Failure
5214,254016,1,P1025598-00-U-ST14I0057916R1,P1025598-00-U:ST14I0057916R1,1025598-00-U,2024-06-14 07:06:38,6,14,2024,7,6,6208 Cage Failure,1,53.423180,24,P1025598-00-U:ST14I0057916R1+6208 Cage Failure
5260,254019,1,P1025598-01-U-ST14C0038654R1,P1025598-01-U:ST14C0038654R1,1025598-01-U,2024-06-14 07:17:04,6,14,2024,7,17,6208 Cage Failure,1,54.538368,24,P1025598-01-U:ST14C0038654R1+6208 Cage Failure
5904,254061,1,P1025598-00-U-ST13D0013763R1,P1025598-00-U:ST13D0013763R1,1025598-00-U,2024-06-14 11:07:58,6,14,2024,11,7,6208 Cage Failure,1,50.936279,24,P1025598-00-U:ST13D0013763R1+6208 Cage Failure


In [58]:
df_failures_and_Passes = df[df['thing+metricName'].isin(df_allFailures['thing+metricName'].unique())]

In [59]:
failures_and_passes_summary = df_failures_and_Passes.drop_duplicates(['part', 'metricName'])[['part', 'metricName', 'thing+metricName']]
failures_and_passes_summary 

,part,metricName,thing+metricName
1192,P1025598-00-U-ST13B0008007,Intermediate Gear Clicking,P1025598-00-U:ST13B0008007+Intermediate Gear C...
1199,P1025598-00-U-ST13B0008007,Intermediate Gear Clicking Negative,P1025598-00-U:ST13B0008007+Intermediate Gear C...
1350,P1025598-01-U-ST18F1055208,Motor 3rd Ord Avg,P1025598-01-U:ST18F1055208+Motor 3rd Ord Avg
1352,P1025598-01-U-ST18F1055208,Growl 4-6,P1025598-01-U:ST18F1055208+Growl 4-6
1442,P1025598-01-U-ST12L0003837,Motor 3rd Ord Avg,P1025598-01-U:ST12L0003837+Motor 3rd Ord Avg
1444,P1025598-01-U-ST12L0003837,Growl 4-6,P1025598-01-U:ST12L0003837+Growl 4-6
2500,P1025598-01-U-ST13E0014479,Motor 3rd Ord Avg,P1025598-01-U:ST13E0014479+Motor 3rd Ord Avg
2503,P1025598-01-U-ST13E0014479,DI Alert,P1025598-01-U:ST13E0014479+DI Alert
2738,P1025598-01-U-ST14L0073536,Input Gear Clicking Negative,P1025598-01-U:ST14L0073536+Input Gear Clicking...
3215,P1025598-00-U-ST13D0013706,CH/Input Bearing Ball Defect,P1025598-00-U:ST13D0013706+CH/Input Bearing Ba...


# Is there a passing test?


In [60]:
#checks to see if a passing test exists
def isPassTest(part):
    df_tmp = df_failures_and_Passes[(df_failures_and_Passes['part'] == part) & (df_failures_and_Passes['isTestPass'] == 1)]
    if df_tmp.shape[0] == 0:
        return False
    else:
        return True

In [61]:
failures_and_passes_summary['PassingTest?'] = failures_and_passes_summary['part'].apply(isPassTest)

In [62]:
failures_and_passes_summary

,part,metricName,thing+metricName,PassingTest?
1192,P1025598-00-U-ST13B0008007,Intermediate Gear Clicking,P1025598-00-U:ST13B0008007+Intermediate Gear C...,True
1199,P1025598-00-U-ST13B0008007,Intermediate Gear Clicking Negative,P1025598-00-U:ST13B0008007+Intermediate Gear C...,True
1350,P1025598-01-U-ST18F1055208,Motor 3rd Ord Avg,P1025598-01-U:ST18F1055208+Motor 3rd Ord Avg,True
1352,P1025598-01-U-ST18F1055208,Growl 4-6,P1025598-01-U:ST18F1055208+Growl 4-6,True
1442,P1025598-01-U-ST12L0003837,Motor 3rd Ord Avg,P1025598-01-U:ST12L0003837+Motor 3rd Ord Avg,True
1444,P1025598-01-U-ST12L0003837,Growl 4-6,P1025598-01-U:ST12L0003837+Growl 4-6,True
2500,P1025598-01-U-ST13E0014479,Motor 3rd Ord Avg,P1025598-01-U:ST13E0014479+Motor 3rd Ord Avg,True
2503,P1025598-01-U-ST13E0014479,DI Alert,P1025598-01-U:ST13E0014479+DI Alert,True
2738,P1025598-01-U-ST14L0073536,Input Gear Clicking Negative,P1025598-01-U:ST14L0073536+Input Gear Clicking...,True
3215,P1025598-00-U-ST13D0013706,CH/Input Bearing Ball Defect,P1025598-00-U:ST13D0013706+CH/Input Bearing Ba...,True


In [63]:
def calculateTime(part): #If there is a successful rework, find the time between the latest success and the latest failure
    df_test = df_failures_and_Passes[df_failures_and_Passes['part'] == part].drop_duplicates(['testID', 'isTestPass']).reset_index()
    if 1 in df_test['isTestPass'].values:
        passingLoc = df_test[df_test['isTestPass'] == 1].index
        comparison_result = df_test.loc[passingLoc[0], 'testTime'] - df_test.loc[passingLoc[0] - 1, 'testTime']
        return comparison_result
    else: return np.nan

In [64]:
def failDisposition(row):
    if row['MachineHealth?'] == True:
        return 'Machine Health Failure'
    elif row['retestSuccess?'] == True:
        return 'Retest Success'
    elif row['PassingTest?'] == True:
        return 'Reworked Pass'
    else:
        return 'Pending Rework / Unsuccessful Rework'
    

In [65]:
#cleaningTable
failures_and_passes_summary['TimeToPass'] = failures_and_passes_summary['part'].apply(calculateTime)
failures_and_passes_summary['TimeToPass_minutes'] = failures_and_passes_summary['TimeToPass'].apply(lambda x: x.total_seconds()/60)
failures_and_passes_summary['retestSuccess?'] = failures_and_passes_summary['TimeToPass_minutes'].apply(lambda x: True if x < retestTime else False)
failures_and_passes_summary['MachineHealth?'] = failures_and_passes_summary['metricName'].apply(lambda x: False if x in NVH_metrics else True)

#create a column to disposition each part based on passingTest? retestSuccess? and machine Health?
failures_and_passes_summary['disposition'] = failures_and_passes_summary.apply(failDisposition, axis = 1)
failures_and_passes_summary

,part,metricName,thing+metricName,PassingTest?,TimeToPass,TimeToPass_minutes,retestSuccess?,MachineHealth?,disposition
1192,P1025598-00-U-ST13B0008007,Intermediate Gear Clicking,P1025598-00-U:ST13B0008007+Intermediate Gear C...,True,0 days 03:21:28,201.466667,False,False,Reworked Pass
1199,P1025598-00-U-ST13B0008007,Intermediate Gear Clicking Negative,P1025598-00-U:ST13B0008007+Intermediate Gear C...,True,0 days 03:21:28,201.466667,False,False,Reworked Pass
1350,P1025598-01-U-ST18F1055208,Motor 3rd Ord Avg,P1025598-01-U:ST18F1055208+Motor 3rd Ord Avg,True,0 days 00:05:30,5.500000,True,False,Retest Success
1352,P1025598-01-U-ST18F1055208,Growl 4-6,P1025598-01-U:ST18F1055208+Growl 4-6,True,0 days 00:05:30,5.500000,True,False,Retest Success
1442,P1025598-01-U-ST12L0003837,Motor 3rd Ord Avg,P1025598-01-U:ST12L0003837+Motor 3rd Ord Avg,True,0 days 00:05:28,5.466667,True,False,Retest Success
1444,P1025598-01-U-ST12L0003837,Growl 4-6,P1025598-01-U:ST12L0003837+Growl 4-6,True,0 days 00:05:28,5.466667,True,False,Retest Success
2500,P1025598-01-U-ST13E0014479,Motor 3rd Ord Avg,P1025598-01-U:ST13E0014479+Motor 3rd Ord Avg,True,0 days 00:05:04,5.066667,True,False,Retest Success
2503,P1025598-01-U-ST13E0014479,DI Alert,P1025598-01-U:ST13E0014479+DI Alert,True,0 days 00:05:04,5.066667,True,True,Machine Health Failure
2738,P1025598-01-U-ST14L0073536,Input Gear Clicking Negative,P1025598-01-U:ST14L0073536+Input Gear Clicking...,True,0 days 00:05:44,5.733333,True,False,Retest Success
3215,P1025598-00-U-ST13D0013706,CH/Input Bearing Ball Defect,P1025598-00-U:ST13D0013706+CH/Input Bearing Ba...,True,0 days 00:05:29,5.483333,True,False,Retest Success


In [66]:
dispositionCounts = failures_and_passes_summary.drop_duplicates(subset = 'part')['disposition'].value_counts()

In [67]:
for index, values in dispositionCounts.iteritems():
    summary[index] = values

C:\Users\Edmund\AppData\Local\Temp\ipykernel_12536\3669781417.py:1: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, values in dispositionCounts.iteritems():


In [68]:
summary

FirstPasses                             94
DLE                                      7
Retest Success                           8
Reworked Pass                            0
Pending Rework / Unsuccessful Rework     0
Machine Health                           0
Reworked Pass                            2
dtype: int64